##IMPORTS

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Imports de Regresión Lineal
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

##CARGAR Y USAR DATASET

In [2]:
df = pd.read_csv("Entradas_de_extranjeros_a_Colombia_20250430.csv")
print(df)

         Año        Mes             Nacionalidad Codigo Iso 3166 Femenino  \
0       2012      Enero  Islas Alboran y Perejil          99,999        1   
1       2012      Enero                  Albania               8        1   
2       2012      Enero                 Alemania             276       11   
3       2012      Enero                 Alemania             276        1   
4       2012      Enero                 Alemania             276       63   
...      ...        ...                      ...             ...      ...   
158212  2024  Diciembre              Desconocido          99,999        0   
158213  2024  Diciembre              Desconocido          99,999       59   
158214  2024  Diciembre              Desconocido          99,999        1   
158215  2024  Diciembre              Desconocido          99,999        0   
158216  2024  Diciembre              Desconocido          99,999        1   

       Masculino  Indefinido Total      Latitud - Longitud  
0             

##EDA (EXPLORACIÓN INICIAL DE LOS DATOS)

In [3]:
df.columns

Index(['Año', 'Mes', 'Nacionalidad', 'Codigo Iso 3166', 'Femenino',
       'Masculino', 'Indefinido', 'Total', 'Latitud - Longitud'],
      dtype='object')

In [4]:
df.describe()

,Año,Indefinido
count,158217.000000,0.0
mean,2018.148208,NaN
std,3.852461,NaN
min,2012.000000,NaN
25%,2015.000000,NaN
50%,2018.000000,NaN
75%,2022.000000,NaN
max,2024.000000,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158217 entries, 0 to 158216
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Año                 158217 non-null  int64  
 1   Mes                 158217 non-null  object 
 2   Nacionalidad        158217 non-null  object 
 3   Codigo Iso 3166     158217 non-null  object 
 4   Femenino            158217 non-null  object 
 5   Masculino           158217 non-null  object 
 6   Indefinido          0 non-null       float64
 7   Total               158217 non-null  object 
 8   Latitud - Longitud  158217 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 10.9+ MB


In [6]:
df.isnull().sum()

,0
Año,0
Mes,0
Nacionalidad,0
Codigo Iso 3166,0
Femenino,0
Masculino,0
Indefinido,158217
Total,0
Latitud - Longitud,0


In [7]:
df.duplicated().sum()

np.int64(46)

In [8]:
df.head(7)

,Año,Mes,Nacionalidad,Codigo Iso 3166,Femenino,Masculino,Indefinido,Total,Latitud - Longitud
0,2012,Enero,Islas Alboran y Perejil,"99,999",1,0,NaN,1,"(4.697144,-74.140516)"
1,2012,Enero,Albania,8,1,2,NaN,3,"(4.697144,-74.140516)"
2,2012,Enero,Alemania,276,11,16,NaN,27,"(-4.197703,-69.941278)"
3,2012,Enero,Alemania,276,1,1,NaN,2,"(0.25129,-76.875963)"
4,2012,Enero,Alemania,276,63,102,NaN,165,"(0.814836,-77.662532)"
5,2012,Enero,Alemania,276,22,23,NaN,45,"(10.408582,-75.538003)"
6,2012,Enero,Alemania,276,27,48,NaN,75,"(10.445761,-75.516429)"


##PRE PROCESAMIENTO DE LOS DATOS

In [9]:
#Convertir el Codigo Iso 3166 de string a float
df['Codigo Iso 3166'] = df['Codigo Iso 3166'].str.replace(',', '').astype(float)

In [10]:
#Convertir total a entero quitando las comas
df['Total'] = df['Total'].str.replace(',', '').astype(int)

In [11]:
#Convertir masculino y femenino a entero
df['Masculino'] = df['Masculino'].str.replace(',', '').astype(int)
df['Femenino'] = df['Femenino'].str.replace(',', '').astype(int)


In [12]:
#Convertir los meses a 1,2,3,4,5,6,7,8,9,10,11,12 dependiendo del mes
df['Mes'] = df['Mes'].replace({'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12})

<ipython-input-12-bf5365217790>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Mes'] = df['Mes'].replace({'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12})


In [13]:
#Eliminar columna nacionalidad ya que tiene muchos registros diferentes
df.drop('Nacionalidad', axis=1, inplace=True)

In [14]:
df.sample(5)

,Año,Mes,Codigo Iso 3166,Femenino,Masculino,Indefinido,Total,Latitud - Longitud
78342,2018,5,826.0,92,238,NaN,330,"(6.171601,-75.427454)"
70574,2017,10,660.0,0,1,NaN,1,"(4.697144,-74.140516)"
2731,2012,3,826.0,0,4,NaN,4,"(8.635765,-77.346578)"
36739,2015,2,300.0,40,67,NaN,107,"(4.697144,-74.140516)"
77227,2018,4,826.0,4,4,NaN,8,"(0.25129,-76.875963)"


In [15]:
#Dividir Latitud - Longitud en 2 columnas diferentes
# 1. Eliminar paréntesis
df['Latitud - Longitud'] = df['Latitud - Longitud'].str.replace('[()]', '', regex=True)

# 2. Solo dividir si hay una coma (para evitar errores en "No Aplica")
def split_lat_lon(value):
    if ',' in value:
        return value.split(',')
    else:
        return [np.nan, np.nan]

df[['Latitud', 'Longitud']] = df['Latitud - Longitud'].apply(split_lat_lon).apply(pd.Series)

# 3. Convertir a float ignorando errores
df['Latitud'] = pd.to_numeric(df['Latitud'], errors='coerce')
df['Longitud'] = pd.to_numeric(df['Longitud'], errors='coerce')

# Mostrar resultado
print(df)

         Año  Mes  Codigo Iso 3166  Femenino  Masculino  Indefinido  Total  \
0       2012    1          99999.0         1          0         NaN      1   
1       2012    1              8.0         1          2         NaN      3   
2       2012    1            276.0        11         16         NaN     27   
3       2012    1            276.0         1          1         NaN      2   
4       2012    1            276.0        63        102         NaN    165   
...      ...  ...              ...       ...        ...         ...    ...   
158212  2024   12          99999.0         0          2         NaN      2   
158213  2024   12          99999.0        59         63         NaN    122   
158214  2024   12          99999.0         1          3         NaN      4   
158215  2024   12          99999.0         0          1         NaN      1   
158216  2024   12          99999.0         1          1         NaN      2   

          Latitud - Longitud   Latitud   Longitud  
0        4.

In [16]:
df.isnull().sum()

,0
Año,0
Mes,0
Codigo Iso 3166,0
Femenino,0
Masculino,0
Indefinido,158217
Total,0
Latitud - Longitud,0
Latitud,14
Longitud,14


In [17]:
#Eliminar columna indefinido ya que todos sus valores son nulos
df.drop('Indefinido', axis=1, inplace=True)

In [18]:
#Eliminar filas nulas de latitud y longitud
df.dropna(subset=['Latitud', 'Longitud'], inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158203 entries, 0 to 158216
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Año                 158203 non-null  int64  
 1   Mes                 158203 non-null  int64  
 2   Codigo Iso 3166     158203 non-null  float64
 3   Femenino            158203 non-null  int64  
 4   Masculino           158203 non-null  int64  
 5   Total               158203 non-null  int64  
 6   Latitud - Longitud  158203 non-null  object 
 7   Latitud             158203 non-null  float64
 8   Longitud            158203 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 12.1+ MB


In [20]:
#Eliminar columna latitud-longitud
df.drop('Latitud - Longitud', axis=1, inplace=True)

In [21]:
#Eliminar columnas femenino y masculino
df.drop(['Femenino', 'Masculino'], axis=1, inplace=True)

#Normalizar variables


In [22]:
#Normalizar variables
scaler = StandardScaler()
df[['Codigo Iso 3166', 'Mes', 'Año', 'Latitud', 'Longitud']] = scaler.fit_transform(df[['Codigo Iso 3166', 'Mes', 'Año', 'Latitud', 'Longitud']])

In [23]:
df.head()

,Año,Mes,Codigo Iso 3166,Total,Latitud,Longitud
0,-1.595934,-1.569338,11.753030,1,-0.461038,0.380414
1,-1.595934,-1.569338,-0.134772,3,-0.461038,0.380414
2,-1.595934,-1.569338,-0.102910,27,-2.661225,2.148967
3,-1.595934,-1.569338,-0.102910,2,-1.560743,-0.771648
4,-1.595934,-1.569338,-0.102910,165,-1.421347,-1.102920


##SEPARAR DATOS DE ENTRENAMIENTOO Y PRUEBA

In [27]:
#Separar los datos
x = df.drop(columns = ['Total']) #VARIABLE OBJETIVO
y = df['Total'] #Variable dependiente (lo que queremos predecir)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

##ENTRENAR MODELO

In [28]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

##EVALUAR MODELO

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#Definir el modelo con hiperparámetros básicos
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

#Entrenar el modelo
rf_model.fit(x_train, y_train)

#Hacer predicciones
y_pred_rf = rf_model.predict(x_test)

#Evaluar el modelo
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

#Mostrar los resultados
print(f'MSE: {mse_rf}')
print(f'MAE: {mae_rf}')
print(f'R2: {r2_rf}')

MSE: 96375.56600597741
MAE: 41.931396220920256
R2: 0.9573375525302938


##HACER UNA PREDICCIÓN

In [36]:
import pandas as pd

# Crear diccionario nuevo
nuevo = {
    'Codigo Iso 3166': [862],
    'Mes': [7],
    'Año': [2025],
    'Latitud': [10.4806],
    'Longitud': [-66.9036]
}

nuevo_df = pd.DataFrame(nuevo)

# Aplicar el scaler entrenado
nuevo_df[['Codigo Iso 3166', 'Mes', 'Año', 'Latitud', 'Longitud']] = scaler.transform(
    nuevo_df[['Codigo Iso 3166', 'Mes', 'Año', 'Latitud', 'Longitud']]
)

# Reordenar columnas para que coincidan con x_train
nuevo_df = nuevo_df.reindex(columns=x_train.columns)


# Predecir con el modelo entrenado (rf_model)
prediccion = rf_model.predict(nuevo_df)
total_migrantes = int(round(prediccion[0]))
print(f'El total de migrantes que ingresarían son: {total_migrantes:,}')


El total de migrantes que ingresarían son: 638
